In [20]:
# pip install sktime

In [21]:
import pandas as pd
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
from sktime.forecasting.compose import make_reduction
from sklearn.exceptions import ConvergenceWarning
import requests
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    ExpandingWindowSplitter
)
from sktime.forecasting.compose import MultiplexForecaster
from sklearn.neighbors import KNeighborsRegressor
from sktime.forecasting.ets import AutoETS
from sktime.transformations.series.boxcox import LogTransformer


import warnings
# warnings.filterwarnings('ignore')

In [30]:
def initialize_arima_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    residual_forecaster = ARIMA()

    # Create the TransformedTargetForecaster pipeline
    pipe = TransformedTargetForecaster(
        [
            ("log_transformer", LogTransformer()),
            ("deseasonalizer_daily", deseasonalizer_daily),
            ("residual_forecaster", residual_forecaster),
        ]
    )

    return pipe

def initialize_elasticnet_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")

    # Create the TransformedTargetForecaster pipeline
    pipe = TransformedTargetForecaster(
    [
        ("log_transformer", LogTransformer()),
        ("deseasonalizer_daily", deseasonalizer_daily),
        ("forecast", make_reduction(
                ElasticNetCV(n_jobs=-1),
                window_length=24,
                strategy="direct",
            )
        )
    ]
    )

    return pipe

def initialize_rf_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")

    # Create the TransformedTargetForecaster pipeline
    pipe = TransformedTargetForecaster(
        [
            ("log_transformer", LogTransformer()),
            ("deseasonalizer_daily", deseasonalizer_daily),
            ("forecast", make_reduction(
                RandomForestRegressor(n_estimators=200, n_jobs=-1),
                window_length=24,
                strategy="direct",
            )
            )
        ]
    )

    return pipe

def initialize_gb_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = TransformedTargetForecaster(
        [

            ("log_transformer", LogTransformer()),
            ("deseasonalizer_daily", deseasonalizer_daily),
            ("forecast", make_reduction(
                GradientBoostingRegressor(n_estimators=200),
                window_length=24,
                strategy="direct",
            )
        )
        ]
    )

    return pipe


def initialize_hist_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = TransformedTargetForecaster(
        [
            ("log_transformer", LogTransformer()),
            ("deseasonalizer_daily", deseasonalizer_daily),
            ("forecast", make_reduction(
                HistGradientBoostingRegressor(),
                window_length=24,
                strategy="direct",
            )
        )
        ]
    )

    return pipe

def initialize_ets_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = TransformedTargetForecaster(
        [
            ("log_transformer", LogTransformer()),
            ("deseasonalizer_daily", deseasonalizer_daily),
            ("forecast", make_reduction(
                AutoETS(auto=True,sp=24, n_jobs=-1),
                window_length=24,
                strategy="direct",
            )
        )
        ]
    )

    return pipe

from sktime.transformations.series.boxcox import BoxCoxTransformer
from lightgbm import LGBMRegressor

def initialize_lgbm_forecaster():
    pipe = TransformedTargetForecaster(
                    [
                        ("log_transformer", BoxCoxTransformer()),
                        (
                            "deseasonalizer_weekly",
                            Deseasonalizer(sp=24 * 7, model="additive"),
                        ),
                        (
                            "forecast",
                            make_reduction(
                                LGBMRegressor(),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                )

    return pipe

In [32]:
arima_pipeline = initialize_arima_forecaster()
elasticnet_pipeline = initialize_elasticnet_forecaster()
rf_pipeline = initialize_rf_forecaster()
gb_pipeline = initialize_gb_forecaster()
hist_pipeline = initialize_hist_forecaster()
ets_pipeline = initialize_ets_forecaster()
lgbm_pipeline = initialize_lgbm_forecaster()

forecasting_models = {
    'elasticnet_pipeline': elasticnet_pipeline,
    'rf_pipeline': rf_pipeline,
    'gb_pipeline': gb_pipeline,
    'hist_pipeline': hist_pipeline,
    'ets_pipeline': ets_pipeline
}

forecaster = MultiplexForecaster(
    forecasters=[
        ("elasticnet_pipeline", elasticnet_pipeline),
        ("rf_pipeline", rf_pipeline),
        ("gb_pipeline", gb_pipeline),
        ("hist_pipeline", hist_pipeline),
        ("ets_pipeline", ets_pipeline),
        ("lgbm_pipeline", lgbm_pipeline)
    ]
)

forecaster_param_grid = {"selected_forecaster": ["elasticnet_pipeline", "rf_pipeline", "gb_pipeline", "hist_pipeline"]}

c:\Users\rkris\miniconda3\envs\slalomenv\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1480: UserWarning: The `scitype` of the given `estimator` cannot be inferred. Assuming "tabular-regressor" = scikit-learn regressor interface. If this warning is followed by an unexpected exception, please consider report as a bug on the sktime issue tracker.
  warn(


In [33]:
price_old_df = pd.read_csv('https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
price_old_df = price_old_df.sort_values(by='Date (MST)')
price_old_df = price_old_df.asfreq('H')

price_df = price_old_df.loc["2023-01-01":"2023-03-28", ["price"]]

# y_train, y_test = temporal_train_test_split(price_df, test_size=48)

In [37]:
cv = ExpandingWindowSplitter(initial_window=int(len(price_df) * 0.85), step_length=12, fh=np.arange(1, 13))
n_splits = cv.get_n_splits(price_df)
print(f"Number of Folds = {n_splits}")

Number of Folds = 26


In [26]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredScaledError, MeanSquaredError

list_models = ["rf_pipeline"]

rmse_cv_results = []
rmse_cv_std = []
for i in list_models:
    forecaster.set_params(**{"selected_forecaster": i})
    results = evaluate(
        forecaster=forecaster, y=price_df, cv=cv, strategy="refit", return_data=True, scoring=MeanSquaredError(square_root=True), backend="loky"
    )
    rmse = results["test_MeanSquaredError"].mean()
    rmse_std = results["test_MeanSquaredError"].std()
    rmse_cv_results.append(rmse)
    rmse_cv_std.append(rmse_std)

In [28]:
rmse_cv_results_df = pd.DataFrame(
    {"Model": list_models, "RMSE_CV": rmse_cv_results, "RMSE_CV_STD": rmse_cv_std}
).sort_values(by=["RMSE_CV"])
rmse_cv_results_df

,Model,RMSE_CV,RMSE_CV_STD
0,rf_pipeline,168.927732,101.26666


In [38]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredScaledError, MeanSquaredError

list_models = ["lgbm_pipeline"]

rmse_cv_results = []
rmse_cv_std = []
for i in list_models:
    forecaster.set_params(**{"selected_forecaster": i})
    results = evaluate(
        forecaster=forecaster, y=price_df, cv=cv, strategy="refit", return_data=True, scoring=MeanSquaredError(square_root=True), backend="loky"
    )
    rmse = results["test_MeanSquaredError"].mean()
    rmse_std = results["test_MeanSquaredError"].std()
    rmse_cv_results.append(rmse)
    rmse_cv_std.append(rmse_std)

In [39]:
rmse_cv_results_df = pd.DataFrame(
    {"Model": list_models, "RMSE_CV": rmse_cv_results, "RMSE_CV_STD": rmse_cv_std}
).sort_values(by=["RMSE_CV"])
rmse_cv_results_df

,Model,RMSE_CV,RMSE_CV_STD
0,lgbm_pipeline,170.906891,117.758127
